# 1) RNN 기초

220603

## Naive Neural Networks 의 문제점

```
Yesterday, Harry Potter met Hermione Granger
```

- 사람 이름 = 1
- 아니라면 = 0

![](https://i.imgur.com/zPTrFRt.png)

만약 아래 문장이라면 위의 구조로 가능할까?

```
First time Hermione meet Harry and Ron
```

#### 기계 번역 문제의 경우, 입력과 출력 시퀀스의 길이가 동일하지 않을수 있다!

## Recurrent Neural Networks (RNN)

![](https://i.imgur.com/owxFJp3.png)

### 장점

- RNN Cell을 여러개 연결하여 유연한 구조를 만든다.
- y^<2> 예측 시 x<2>만 사용하는 것이 아니라, 첫번째 단어로 연산한 정보의 일부(a<1>)를 사용하게 된다.

### 기본적인 RNN 단점

- 앞서 나온 단어들의 정보만을 사용한다.
    - `He said, "Teddy Roosevelt was a great President."`
    - `He said, "Teddy bears are on sale."`
- Gradient vanishing
    - `The cat, which already ate ............, was full`
    - `The cats, which already ate ............, were full`
    - 신경망이 깊어질수록 (문장이 길어질수록) 문장 초반부의 단어가 후반부 단어에 미치는 영향이 적어진다. (가까이 있는 입력 영향을 더 많이 받음)

## RNN 발전형

### RNN

![](https://i.imgur.com/vcUIhTL.png)

### Long Short-Term Memory (LSTM)

![](https://i.imgur.com/VXeCc8x.png)

LSTM은 메모리를 컨베이어 벨트같이 오래 전파하는 특성을 가진다.

### Gated Recurrent Unit (GRU)

![](https://i.imgur.com/fXeGJu6.png)

- 일반적으로 GRU는 LSTM보다 학습 속도가 빠르다.
- 여러 평가에서 GRU는 LSTM과 비슷한 성능을 보인다.
- 데이터 양이 적을 때는 GRU를 사용하는 것이 좋다.

# 2) RNN 기반 텍스트 처리

220603

- [ref](https://github.com/graykode/nlp-tutorial)

In [1]:
# code by Tae Hwan Jung @graykode
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

def make_batch():
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()  # space tokenizer
        input = [word_dict[n] for n in word[:-1]]  # create (1~n-1) as input
        target = word_dict[word[-1]]  # create (n) as target, We usually call this 'casual language model'

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
#         self.rnn = nn.GRU(input_size=n_class, hidden_size=n_hidden)        
        
        self.W = nn.Linear(n_hidden, n_class, bias=False)
        self.b = nn.Parameter(torch.ones([n_class]))

    def forward(self, hidden, X):
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class]
        outputs, hidden = self.rnn(X, hidden)
        # outputs : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        outputs = outputs[-1] # [batch_size, num_directions(=1) * n_hidden]
        model = self.W(outputs) + self.b # model : [batch_size, n_class]
        return model

In [2]:
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
batch_size = len(sentences)

model = TextRNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)


In [3]:
# Training
for epoch in range(5000):
    optimizer.zero_grad()

    # hidden : [num_layers * num_directions, batch, hidden_size]
    hidden = torch.zeros(1, batch_size, n_hidden)
    # input_batch : [batch_size, n_step, n_class]
    output = model(hidden, input_batch)

    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

input = [sen.split()[:2] for sen in sentences]

# Predict
hidden = torch.zeros(1, batch_size, n_hidden)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 0.193783
Epoch: 2000 cost = 0.022608
Epoch: 3000 cost = 0.007388
Epoch: 4000 cost = 0.003477
Epoch: 5000 cost = 0.001853
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
